# ML Pipeline

**Course: Applied AI in Chemical and Process Engineering**

This notebook witll build a pipeline to prepare data and develop an ML algorithm

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.optimize import minimize
from sklearn.impute import SimpleImputer

# 1. Data Preperation


In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/dissabnd/Applied-AI-in-Chemical-and-Process-Engineering/refs/heads/main/data/reactor.csv')

In [8]:
# prompt: list all columns

df.columns

Index(['Temperature (°C)', 'Pressure (atm)', 'Catalyst Concentration (wt%)',
       'Feed Flow Rate (L/min)', 'Yield (%)', 'Recorded By'],
      dtype='object')

In [10]:
# Print data table
df.head(10)

,Temperature (°C),Pressure (atm),Catalyst Concentration (wt%),Feed Flow Rate (L/min),Yield (%),Recorded By
0,205.0,5.00,2.4,9.50,70.00,Nuwan
1,200.0,5.00,2.1,9.00,70.00,Mali
2,225.0,6.00,3.0,10.25,75.75,Nuwan
3,200.0,6.00,2.4,10.50,70.00,Kavi
4,220.0,6.50,3.0,9.50,95.00,Nuwan
5,210.0,4.00,4.0,11.75,73.42,Janith
6,180.0,5.00,3.1,11.25,70.00,Mali
7,190.0,4.75,4.0,9.25,82.51,Janith
8,220.0,5.50,2.2,10.25,71.12,Janith
9,195.0,6.25,2.9,11.50,70.90,Kavi


In [12]:
# Size of the data table

df.shape

(80, 6)

# Data Quality Check